### Урок 6. Рекуррентные нейронные сети. LSTM. GRU.#

### -- Автор: Шенк Евгений Станиславович

#### Провести анализ как лучше генерировать текст по словам или символам ноутбук text_generation.ipynb

Выполнение:  
Проведено обучение модели для генерации на символах и на словах (в модели отличие только в эпохах: символы-100, слова-200 и BATCH_SIZE: символы-64, слова-16). Для разбиения на слова использован word_tokenize.  
Результат:  
На словах получилось в данной генерации лучше (результат внизу нотбука), но обычно и там и там результат получается средненький.

In [1]:
import tensorflow as tf
from nltk.tokenize import word_tokenize, sent_tokenize

import numpy as np
import os
import time
import itertools

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [2]:
path_to_file = '../data/evgenyi_onegin.txt'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

#### Char

In [4]:
vocab = sorted(set(text))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

#### Word

In [5]:
text_for_words = text.lower().replace('\n', ' \\n ')
text_in_words = word_tokenize(text_for_words, language='russian')
text_in_words = ['\n' if x=='\\n' else x for x in text_in_words]

In [6]:
vocab_words = sorted(set(text_in_words))

word2idx = {u:i for i, u in enumerate(vocab_words)}
idx2word = np.array(vocab_words)

text_in_words_as_int = np.array([word2idx[c] for c in text_in_words])

### train and target

In [7]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# For words
w_examples_per_epoch = len(text_in_words)//(seq_length+1)
w_char_dataset = tf.data.Dataset.from_tensor_slices(text_in_words_as_int)

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
w_sequences = w_char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
for item in sequences.take(1):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [10]:
for item in w_sequences.take(1):
    print(repr(' '.join(idx2word[item.numpy()])))

'александр сергеевич пушкин \n \n евгений онегин \n роман в стихах \n \n не мысля гордый свет забавить , \n вниманье дружбы возлюбя , \n хотел бы я тебе представить \n залог достойнее тебя , \n достойнее души прекрасной , \n святой исполненной мечты , \n поэзии живой и ясной , \n высоких дум и простоты ; \n но так и быть - рукой пристрастной \n прими собранье пестрых глав , \n полусмешных , полупечальных , \n простонародных , идеальных , \n небрежный плод моих забав , \n бессонниц , легких вдохновений , \n незрелых и увядших лет , \n ума'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
w_dataset = w_sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [13]:
for input_example, target_example in  w_dataset.take(1):
    print('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  'александр сергеевич пушкин \n \n евгений онегин \n роман в стихах \n \n не мысля гордый свет забавить , \n вниманье дружбы возлюбя , \n хотел бы я тебе представить \n залог достойнее тебя , \n достойнее души прекрасной , \n святой исполненной мечты , \n поэзии живой и ясной , \n высоких дум и простоты ; \n но так и быть - рукой пристрастной \n прими собранье пестрых глав , \n полусмешных , полупечальных , \n простонародных , идеальных , \n небрежный плод моих забав , \n бессонниц , легких вдохновений , \n незрелых и увядших лет , \n'
Target data: 'сергеевич пушкин \n \n евгений онегин \n роман в стихах \n \n не мысля гордый свет забавить , \n вниманье дружбы возлюбя , \n хотел бы я тебе представить \n залог достойнее тебя , \n достойнее души прекрасной , \n святой исполненной мечты , \n поэзии живой и ясной , \n высоких дум и простоты ; \n но так и быть - рукой пристрастной \n прими собранье пестрых глав , \n полусмешных , полупечальных , \n простонародных , идеальных , \

In [14]:
# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

BATCH_SIZE = 64
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

BATCH_SIZE = 16
w_dataset = w_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

w_dataset

<BatchDataset shapes: ((16, 100), (16, 100)), types: (tf.int32, tf.int32)>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
w_vocab_size = len(vocab_words)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
BATCH_SIZE = 64

model = build_model(
                    vocab_size=vocab_size,
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=BATCH_SIZE)

In [18]:
BATCH_SIZE = 16

model_for_words = build_model(
                    vocab_size=w_vocab_size,
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=BATCH_SIZE)

### Train the model

In [19]:
# Directory where the checkpoints will be saved
char_checkpoint_dir = './training_checkpoints/char'
word_checkpoint_dir = './training_checkpoints/word'
# Name of the checkpoint files
char_checkpoint_prefix = os.path.join(char_checkpoint_dir, "ckpt_{epoch}")
word_checkpoint_prefix = os.path.join(word_checkpoint_dir, "ckpt_{epoch}")

char_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=char_checkpoint_prefix,
    save_weights_only=True)
word_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=word_checkpoint_prefix,
    save_weights_only=True)

In [20]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [21]:
model.compile(optimizer='adam', loss=loss)
model_for_words.compile(optimizer='adam', loss=loss)

In [22]:
EPOCHS = 100

In [23]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[char_checkpoint_callback])

Epoch 1/100
44/44 [==============================] - 4s 98ms/step - loss: 2.3452
Epoch 2/100
44/44 [==============================] - 4s 98ms/step - loss: 1.6894
Epoch 3/100
44/44 [==============================] - 4s 96ms/step - loss: 1.4785
Epoch 4/100
44/44 [==============================] - 4s 96ms/step - loss: 1.3702
Epoch 5/100
44/44 [==============================] - 4s 97ms/step - loss: 1.3150: 0s - loss: 1.315
Epoch 6/100
44/44 [==============================] - 4s 98ms/step - loss: 1.2794
Epoch 7/100
44/44 [==============================] - 4s 96ms/step - loss: 1.2469
Epoch 8/100
44/44 [==============================] - 4s 97ms/step - loss: 1.2204
Epoch 9/100
44/44 [==============================] - 4s 98ms/step - loss: 1.2018
Epoch 10/100
44/44 [==============================] - 4s 97ms/step - loss: 1.1742
Epoch 11/100
44/44 [==============================] - 4s 96ms/step - loss: 1.1480
Epoch 12/100
44/44 [==============================] - 4s 100ms/step - loss: 1.1276
Epoch 

44/44 [==============================] - 4s 101ms/step - loss: 0.1702
Epoch 100/100
44/44 [==============================] - 4s 100ms/step - loss: 0.1612


In [24]:
EPOCHS = 200

In [25]:
history_for_words = model_for_words.fit(w_dataset, epochs=EPOCHS, callbacks=[word_checkpoint_callback])

Epoch 1/200
22/22 [==============================] - 3s 120ms/step - loss: 7.2841
Epoch 2/200
22/22 [==============================] - 3s 119ms/step - loss: 6.2822
Epoch 3/200
22/22 [==============================] - 3s 121ms/step - loss: 5.9630
Epoch 4/200
22/22 [==============================] - 3s 120ms/step - loss: 5.6641
Epoch 5/200
22/22 [==============================] - 3s 120ms/step - loss: 5.3848 1s
Epoch 6/200
22/22 [==============================] - 3s 122ms/step - loss: 5.1396
Epoch 7/200
22/22 [==============================] - 3s 122ms/step - loss: 4.8879
Epoch 8/200
22/22 [==============================] - 3s 118ms/step - loss: 4.6237
Epoch 9/200
22/22 [==============================] - 3s 119ms/step - loss: 4.3579 0s - loss: 
Epoch 10/200
22/22 [==============================] - 3s 121ms/step - loss: 4.0689
Epoch 11/200
22/22 [==============================] - 3s 120ms/step - loss: 3.7623
Epoch 12/200
22/22 [==============================] - 3s 120ms/step - loss: 3.438

22/22 [==============================] - 3s 120ms/step - loss: 0.0403
Epoch 99/200
22/22 [==============================] - 3s 121ms/step - loss: 0.0403
Epoch 100/200
22/22 [==============================] - 3s 121ms/step - loss: 0.0396
Epoch 101/200
22/22 [==============================] - 3s 120ms/step - loss: 0.0397
Epoch 102/200
22/22 [==============================] - 3s 120ms/step - loss: 0.0394
Epoch 103/200
22/22 [==============================] - 3s 119ms/step - loss: 0.0400
Epoch 104/200
22/22 [==============================] - 3s 119ms/step - loss: 0.0403
Epoch 105/200
22/22 [==============================] - 3s 120ms/step - loss: 0.0383
Epoch 106/200
22/22 [==============================] - 3s 118ms/step - loss: 0.0403
Epoch 107/200
22/22 [==============================] - 3s 120ms/step - loss: 0.0387
Epoch 108/200
22/22 [==============================] - 3s 118ms/step - loss: 0.0383
Epoch 109/200
22/22 [==============================] - 3s 120ms/step - loss: 0.0368
Epoch 1

22/22 [==============================] - 3s 117ms/step - loss: 0.0537
Epoch 195/200
22/22 [==============================] - 3s 120ms/step - loss: 0.0459
Epoch 196/200
22/22 [==============================] - 3s 121ms/step - loss: 0.0413
Epoch 197/200
22/22 [==============================] - 3s 120ms/step - loss: 0.0360
Epoch 198/200
22/22 [==============================] - 3s 120ms/step - loss: 0.0352
Epoch 199/200
22/22 [==============================] - 3s 119ms/step - loss: 0.0346
Epoch 200/200
22/22 [==============================] - 3s 120ms/step - loss: 0.0362


### Generate

In [26]:
def generate_text(model, start_string, num_generate = 500, unit='char'):  # word
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = num_generate

    # Converting our start string to numbers (vectorizing)
    if unit=='char':
        input_eval = [char2idx[s] for s in start_string]
    elif unit=='word':
        start_string_words = [w for w in start_string.lower().split(' ') if w.strip() != '' or w == '\n']
        input_eval = [word2idx[s] for s in start_string_words]
    else:
        print("Unit != 'char' or 'word'")
        return ""
    
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        if unit=='char':
            text_generated.append(idx2char[predicted_id])
        elif unit=='word':
            text_generated.append(idx2word[predicted_id])          
        else:
            print("Unit != 'char' or 'word'")
            return ""
    
    if unit=='word':
        return (start_string + ' '.join(text_generated))          
    
    return (start_string + ''.join(text_generated))

In [27]:
tf.train.latest_checkpoint(char_checkpoint_dir)

'./training_checkpoints/char\\ckpt_100'

In [28]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(char_checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [36]:
print(generate_text(model, start_string=u"Уж месяц был ", num_generate=500, unit='char'))

Уж месяц был общий приезжатв.

                                      XI

                        Да, может быть, едет на изумленьем
                        О ножки, полно за столов                    Ответа которой окрыль;
                        Вытомой на рукой,
                        Услышу ль вновь я ваши хоры?
                        Взгляну на дев, как садин, А нет.

                                    XXXII

                        И что ж на самом Пера его,
                        Подруге юности мят


Уж месяц был и свотоено в волных снежен  Моствы, вестей и несет;  
                        Оно своей игрой и пестрой Канет  
                        И быстрый може мечтанье  
                                     Свободно дома принимает,  
                        Трядов наивом стрелой.  
                        Вас голову залоглестится она!  
                        И вот однако ж, проклиная),  
                        Всегда нахмурен, молчалив,  
                        Не знаю, правопистью порой  
                       Или   

                        Уж месяц был общий приезжатв.  

                                      XI  

                        Да, может быть, едет на изумленьем  
                        О ножки, полно за столов                    Ответа которой окрыль;  
                        Вытомой на рукой,  
                        Услышу ль вновь я ваши хоры?  
                        Взгляну на дев, как садин, А нет.  

                                    XXXII  

                        И что ж на самом Пера его,  
                        Подруге юности мят  

In [30]:
tf.train.latest_checkpoint(word_checkpoint_dir)

'./training_checkpoints/word\\ckpt_200'

In [31]:
model_for_words = build_model(w_vocab_size, embedding_dim, rnn_units, batch_size=1)

model_for_words.load_weights(tf.train.latest_checkpoint(word_checkpoint_dir))

model_for_words.build(tf.TensorShape([1, None]))

In [34]:
print(generate_text(model_for_words, start_string=u"Уж месяц был ", num_generate=50, unit='word'))

Уж месяц был печальное устах приношенье дамам веселых долго , что пора без душою ? 
 кто , лучше , 
 блеснул мороз . и рады мы 
 проказам матушки зимы . 
 не радо ей лишь сердце тани . 
 нейдет она зиму встречать , 
 морозной пылью подышать 
 и первым


Уж месяц был темноте хозяйский наши раз плеча же скорей  
 марает он ответ учтивый .  
 никто б не мог ее прекрасной  
 назвать ; но с головы до ног  
 никто бы в ней найти не мог  
 того , что модой самовластной  
 в высоком лондонском кругу  
 зовется vulgаr  

Уж месяц был печальное устах приношенье дамам веселых долго , что пора без душою ?  
 кто , лучше ,  
 блеснул мороз . и рады мы  
 проказам матушки зимы .  
 не радо ей лишь сердце тани .  
 нейдет она зиму встречать ,  
 морозной пылью подышать  
 и первым  